In [ ]:
import vocalpy as voc
from ethograph.features.energy import bandpass_envelope

sound = voc.Sound.read("path/to/audio.wav")
data = sound.data[0]  # first channel, 1-D
rate = sound.samplerate

env_time, envelope = bandpass_envelope(data, rate)

In [ ]:
import vocalpy as voc
from ethograph.features.energy import env_ava

sound = voc.Sound.read("path/to/audio.wav")
data = sound.data[0]  # first channel, 1-D
sample_rate = sound.samplerate

env_time, envelope = env_ava(
    audio_data_1d, sample_rate,
    nperseg=102,
)

In [ ]:
import vocalpy as voc
from vocalseg.dynamic_thresholding import dynamic_threshold_segmentation

sound = voc.Sound.read("path/to/audio.wav")
data = sound.data[0]  # first channel, 1-D
rate = sound.samplerate

results = dynamic_threshold_segmentation(
    data, rate,
    min_level_db=-70.0,
    hop_length_ms=5.0,
    silence_threshold=0.1,
    min_syllable_length_s=0.02,
)

In [ ]:
import audioio as aio
from vocalseg.dynamic_thresholding import dynamic_threshold_segmentation

audio_path = "path/to/audio.wav"
data, sample_rate = aio.load_audio(audio_path)
if data.ndim > 1:
    data = data[:, 0]  # select first channel
rate = float(sample_rate)

results = dynamic_threshold_segmentation(data, rate)

Help on function dynamic_threshold_segmentation in module vocalseg.dynamic_thresholding:

dynamic_threshold_segmentation(vocalization, rate, min_level_db=-80, min_level_db_floor=-40, db_delta=5, n_fft=1024, hop_length_ms=1, win_length_ms=5, ref_level_db=20, pre=0.97, silence_threshold=0.05, min_silence_for_spec=0.1, max_vocal_for_spec=1.0, min_syllable_length_s=0.1, spectral_range=None, verbose=False)
    computes a spectrogram from a waveform by iterating through thresholds
         to ensure a consistent noise level

    Arguments:
        vocalization {[type]} -- waveform of song
        rate {[type]} -- samplerate of datas

    Keyword Arguments:
        min_level_db {int} -- default dB minimum of spectrogram (threshold anything below) (default: {-80})
        min_level_db_floor {int} -- highest number min_level_db is allowed to reach dynamically (default: {-40})
        db_delta {int} -- delta in setting min_level_db (default: {5})
        n_fft {int} -- FFT window size (default: 

In [5]:
import vocalseg
vocalseg.dynamic_threshold_segmentation


AttributeError: module 'vocalseg' has no attribute 'dynamic_threshold_segmentation'

In [ ]:
import audioio as aio

audio_path = "path/to/audio.wav"
data, sample_rate = aio.load_audio(audio_path)
if data.ndim > 1:
    data = data[:, 0]  # select first channel
rate = float(sample_rate)

results = (data, rate)

In [1]:
from dandi.dandiapi import DandiAPIClient

client = DandiAPIClient()
dandiset = client.get_dandiset("000231", "0.220904.1554")

asset = dandiset.get_asset_by_path(
    "sub-KM131/sub-KM131_ses-20180116T184757_behavior+ecephys+image.nwb"
)

from pathlib import Path

dest = Path(r"C:\Users\aksel\Documents\Code\EthoGraph\data2")
dest.mkdir(parents=True, exist_ok=True)

asset.download(str(dest))

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\aksel\\Documents\\Code\\EthoGraph\\data2'